In [178]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, accuracy_score
from sklearn.preprocessing import OneHotEncoder


In [209]:
df_train = pd.read_csv("input/mitbih_train.csv", header=None)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv("input/mitbih_test.csv", header=None)

# enc = OneHotEncoder(sparse=False, categories='auto')
Y = np.array(df_train[187].values)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]
# Y = enc.fit_transform(np.expand_dims(Y,1))

Y_test = np.array(df_test[187].values)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]
# Y_test = enc.fit_transform(np.expand_dims(Y_test,1))


In [210]:
BATCH_SIZE = 200
NUM_RESIDUAL_BLOCKS = 5


In [211]:
class CustomDataset(Dataset):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        return torch.tensor(self.x[index]), torch.tensor(self.y[index])

train_dataset = CustomDataset(X, Y)
val_dataset = CustomDataset(X_test, Y_test)

def load_data(train_dataset, val_dataset):

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
    
    return train_loader, val_loader

train_loader, val_loader = load_data(train_dataset, val_dataset)


In [212]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device_count = torch.cuda.device_count()
# device = 'cpu'

In [213]:
class CNResidual(nn.Module):
    def __init__(self, size):
        super().__init__()
        # layers
        self.c1 = nn.Conv1d(size, size, kernel_size=5, padding=2)
        self.c2 = nn.Conv1d(size, size, kernel_size=5, padding=2)
        self.pool = nn.MaxPool1d(kernel_size=5, stride=2)
        
    def forward(self, x):
        # call layers
        x1 = F.relu(self.c1(x))
        x1 = self.c2(x1)
        return self.pool(F.relu(x+x1))
    
class CNet(nn.Module):

    def __init__(self):
        super().__init__()
        # layers
        self.c1 = nn.Conv1d(1, 32, kernel_size=5, padding=2)
        self.cresid1 = CNResidual(32)
        self.cresid2 = CNResidual(32)
        self.cresid3 = CNResidual(32)
        self.cresid4 = CNResidual(32)
        self.cresid5 = CNResidual(32)
#         self.cresids = []
#         for i in range(NUM_RESIDUAL_BLOCKS):
#             self.cresids.append(CNResidual(32).to(device))
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 5)
        
    def forward(self, x):
        # call layers
        x = self.c1(x)
        x = self.cresid1(x)
        x = self.cresid2(x)
        x = self.cresid3(x)
        x = self.cresid4(x)
        x = self.cresid5(x)
#         for cresid in self.cresids:
#             x = cresid(x)
        x  = x.reshape(-1, 64)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

cnetmodel = CNet()
# if device_count > 1:
#     cnetmodel = nn.DataParallel(cnetmodel)
cnetmodel = cnetmodel.to(device)

In [214]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnetmodel.parameters(), lr=0.01)


In [215]:
def eval(model, val_loader):
    model.eval()

    x_pred = []
    x_true = []
    i = 0
    for x, y in val_loader:
        x=x.to(device).reshape(-1, 1, 187).float()
        x_hat = F.softmax(model(x), dim=1)
        x_hat = torch.argmax(x_hat, dim=1)
        x_pred.extend(x_hat.cpu().numpy())
        x_true.extend(y.long().detach().numpy())
    f = f1_score(y_pred=x_pred, y_true=x_true, average='micro')
    acc = accuracy_score(y_pred=x_pred, y_true=x_true)
    return f, acc


In [216]:
def train(model, train_loader, val_loader, n_epochs):
    model.train()
    for epoch in range(n_epochs):
        train_loss = 0
        i=0
        for x, y in train_loader:
            x=x.to(device).reshape(-1, 1, 187).float()
            y = y.to(device).long()
            y_hat = model(x)
            loss = criterion(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))
        f, acc = eval(model, val_loader)
        print('Epoch: %d \t Validation f: %.2f, acc: %.2f'%(epoch+1, f, acc))

In [ ]:
train(cnetmodel, train_loader, val_loader, 2)


Epoch: 1 	 Training Loss: 0.250771
Epoch: 1 	 Validation f: 0.96, acc: 0.96


In [189]:
x, y =next(iter(train_loader))
x=x.to(device).reshape(-1, 1, 187).float()

In [191]:
torch.max(,1)[1]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0,
        0, 4, 4, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 4, 4, 4, 2, 0, 0,
        0, 4, 4, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 4,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 4, 0, 0, 0, 0, 4, 0, 0, 1, 4, 0, 0, 3, 0, 0, 0, 2, 1, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4,
        0, 2, 0, 0, 0, 0, 0, 0])

In [143]:
x_hat = cnetmodel(x)
x_hat
len(val_loader)

110

In [164]:
for i in cnetmodel.parameters():
    print(i)

Parameter containing:
tensor([[[-5.8568, -6.1711, -5.8834, -6.1824, -5.9507]],

        [[-6.4394, -5.6160, -6.2593, -6.0610, -5.8131]],

        [[-5.8998, -5.9876, -6.2768, -6.0842, -5.6371]],

        [[-5.7051, -5.7786, -6.3336, -6.2835, -6.1359]],

        [[ 5.8973,  6.3959,  5.9637,  5.7360,  5.7638]],

        [[ 6.2744,  6.1930,  6.3402,  6.4094,  6.0246]],

        [[ 6.3598,  6.0129,  6.3069,  6.2853,  5.7926]],

        [[ 5.7316,  6.1191,  5.8355,  5.8868,  5.6498]],

        [[ 6.1574,  5.9466,  6.1783,  5.6802,  6.1053]],

        [[ 5.7986,  6.2233,  5.6198,  6.4315,  6.4018]],

        [[ 5.6753,  5.8872,  6.0117,  6.0255,  5.6282]],

        [[ 6.0107,  6.3712,  6.3399,  6.0373, -6.2819]],

        [[ 6.3255,  6.1377,  5.7518,  5.7813,  6.1114]],

        [[ 5.7607, -5.9698, -5.8336, -5.9381, -6.1583]],

        [[-6.3940, -6.1636,  6.0113,  6.2234,  6.0813]],

        [[-5.9837, -5.7589, -5.6015, -5.6966, -5.7631]],

        [[-6.2110, -5.9635, -6.2162, -5.9876, -6.1